In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.5/636.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.7 MB/s eta 0:00:00
2.18.0-dev20240807


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-08-08 13:52:57--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2024-08-08 13:52:57 (7.79 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-08-08 13:52:57--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
train_file = pd.read_csv(train_file_path, sep='\t', names=['class', 'message'],dtype={'class':'str','message':'str'})
test_file = pd.read_csv(test_file_path, sep='\t', names=['class', 'message'],dtype={'class':'str','message':'str'})

print(train_file,end='\n')
print(test_file,end='\n')

#test_file.info()

     class                                            message
0      ham  ahhhh...just woken up!had a bad dream about u ...
1      ham                           you can never do nothing
2      ham  now u sound like manky scouse boy steve,like! ...
3      ham  mum say we wan to go then go... then she can s...
4      ham  never y lei... i v lazy... got wat? dat day ü ...
...    ...                                                ...
4174   ham  just woke up. yeesh its late. but i didn't fal...
4175   ham  what do u reckon as need 2 arrange transport i...
4176  spam  free entry into our £250 weekly competition ju...
4177  spam  -pls stop bootydelious (32/f) is inviting you ...
4178   ham  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]
     class                                            message
0      ham  i am in hospital da. . i will return home in e...
1      ham         not much, just some textin'. how bout you?
2      ham  i probably won't eat at all today

In [4]:
train_file['class']=train_file['class'].map({'ham':0,'spam':1})
test_file['class']=test_file['class'].map({'ham':0,'spam':1})

print(train_file,end='\n')
print(test_file,end='\n')

      class                                            message
0         0  ahhhh...just woken up!had a bad dream about u ...
1         0                           you can never do nothing
2         0  now u sound like manky scouse boy steve,like! ...
3         0  mum say we wan to go then go... then she can s...
4         0  never y lei... i v lazy... got wat? dat day ü ...
...     ...                                                ...
4174      0  just woke up. yeesh its late. but i didn't fal...
4175      0  what do u reckon as need 2 arrange transport i...
4176      1  free entry into our £250 weekly competition ju...
4177      1  -pls stop bootydelious (32/f) is inviting you ...
4178      0  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]
      class                                            message
0         0  i am in hospital da. . i will return home in e...
1         0         not much, just some textin'. how bout you?
2         0  i probably won't 

In [5]:
train_file.message.str.len().max()

910

In [6]:
def df_to_dataset(dataframe, label_name = 'class', batch_size=64):
  dataframe = dataframe.copy()
  labels = dataframe.pop(label_name)
  dataframe = dataframe.squeeze()
  ds = tf.data.Dataset.from_tensor_slices((dataframe, labels)).batch(batch_size)
  return ds

train_ds = df_to_dataset(train_file)
test_ds= df_to_dataset(test_file)

In [7]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print("Message", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Message b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'
Label 0
Message b'you can never do nothing'
Label 0
Message b'now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?'
Label 0


In [18]:
import re
import string

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(string.punctuation),'')

# vectorization

vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=960)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [19]:
#model
model = tf.keras.Sequential([
  vectorize_layer,
  keras.layers.Embedding(10000, 16),
  keras.layers.Dense(1024, activation = 'relu'),
  keras.layers.Dense(1024, activation = 'relu'),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation = 'sigmoid')])

model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer='adam',
              metrics= [tf.metrics.BinaryAccuracy()])

In [20]:
r= model.fit(train_ds,validation_data=test_ds,epochs=10)

Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 12s 137ms/step - binary_accuracy: 0.8149 - loss: 0.4309 - val_binary_accuracy: 0.8657 - val_loss: 0.3932
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - binary_accuracy: 0.8708 - loss: 0.3785 - val_binary_accuracy: 0.9088 - val_loss: 0.2396
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 136ms/step - binary_accuracy: 0.9391 - loss: 0.1722 - val_binary_accuracy: 0.9634 - val_loss: 0.1125
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 138ms/step - binary_accuracy: 0.9730 - loss: 0.0791 - val_binary_accuracy: 0.9749 - val_loss: 0.0790
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 135ms/step - binary_accuracy: 0.9833 - loss: 0.0441 - val_binary_accuracy: 0.9792 - val_loss: 0.0616
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 135ms/step - binary_accuracy: 0.9901 - loss: 0.1518 - val_binary_accuracy: 0.8657 - val_loss: 60.1408
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 134ms/step - binary_accuracy: 0.8179 - loss: 12.0869 - val_binary_accuracy: 0.9741 - val_loss: 0.0959


In [21]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  model_input = pd.Series([pred_text])
  model_prediction = model.predict(model_input)[0,0]
  if model_prediction > 0.1:
        c = 'spam'
  else:
        c = 'ham'
  prediction = [model_prediction, c]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
[0.00020030775, 'ham']


In [22]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
You passed the challenge. Great job!
